In [5]:
import pandas as pd
import numpy as np
import ast
import mysql.connector
from google.cloud import secretmanager

todo : create a script to dictionary without , to generated json file without , in the last line

In [6]:
import numpy as np
import scipy as sp
from scipy import stats

def jsd(p, q, base=np.e):
    '''
        Implementation of pairwise `jsd` based on  
        https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
    '''
    ## convert to np.array
    p, q = np.asarray(p), np.asarray(q)
    ## normalize p, q to probabilities
    p, q = p/p.sum(), q/q.sum()
    m = 1./2*(p + q)
    return stats.entropy(p,m, base=base)/2. +  stats.entropy(q, m, base=base)/2.

In [14]:
london_file = '/media/krowd/Data/CrossBorderSampled/GlobalTopicModelling-180-sampled/global_london180-reviews-topic-composition_sampled_spark.txt'
dubai_file = '/media/krowd/Data/CrossBorderSampled/GlobalTopicModelling-180-sampled/global-dubai180-reviews-topic-composition_sampled_spark.txt'

In [ ]:
PROJECT_ID = 'alpha-274108'

def access_secret_version(self,project_id, secret_id, version_id):
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """
    # Create the Secret Manager client.
    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = client.secret_version_path(project_id, secret_id, version_id)

    # Access the secret version.
    response = client.access_secret_version(name)
    
    payload = response.payload.data.decode('UTF-8')
    return format(payload)


HOSTNAME = access_secret_version(PROJECT_ID, 'CHOST-A', 'latest'),
USER = access_secret_version(PROJECT_ID, 'CHOST-A-USER', 'latest'),
PASSWORD = access_secret_version(PROJECT_ID, 'CHOST-A-PASS', 'latest'),
DATABASE = 'sydney_zomato_database'


In [15]:
def getConnection(HOSTNAME,USER,PASSWORD,DATABASE):
    mydb = mysql.connector.connect(
                host=HOSTNAME,
                user=USER,
                passwd=PASSWORD,
                database=DATABASE
    )

    mycursor = mydb.cursor()
    
    sql = 'SELECT es.entity_id,e.name,es.global_rank FROM entity_summary es,entity e WHERE es.entity_id = e.id'
    
    mycursor.execute(sql)
    df = pd.DataFrame(mycursor.fetchall())
    return df



In [16]:
dubai_entity_df = getConnection(HOSTNAME,USER,PASSWORD,DATABASE)
london_entity_df = getConnection(HOSTNAME,USER,PASSWORD,DATABASE)

In [17]:
print(dubai_entity_df.shape)
dubai_entity_df.columns = ['id','name','krowdscore']
dubai_entity_df.head()

(8848, 3)


,id,name,krowdscore
0,service_b0a328e786274da4b3863b57020ae5d8,Maestro - Conrad Dubai,0.000000
1,service_3f206fe1259548b992f571207562d104,Cozy Cabana,1.644434
2,service_56b8ed95a12a40f39050ee9043ce5ea3,Zaferan Iranian Restaurant,1.598716
3,service_65ab79bc86b94b5d924db3b07f447940,Seattle's Best Coffee,0.966250
4,service_9f053766fb6d41c3bdc4280ee2783baa,Pizza Hut,0.722925


In [18]:
print(london_entity_df.shape)
london_entity_df.columns = ['id','name','krowdscore']
london_entity_df.head()

(12122, 3)


,id,name,krowdscore
0,service_1549650185304b5c36lo89yl,Chicken Society,3.516503
1,service_15496520197337ksbzgwe8jf,Silverberry Deli & Kitchen,2.499218
2,service_1549652912325zauwy18l69k,Rani,2.097026
3,service_15496516384251dwnb78bc39,Caphe House,3.162269
4,service_1549652006329y3444626wb,Jewel Piccadilly,2.118449


In [19]:
with open(dubai_file) as f:
    data = ast.literal_eval(f.read())
dubai_model = pd.DataFrame(data)
print(dubai_model.shape)
dubai_model = dubai_model.merge(dubai_entity_df,on='id',how='inner')
dubai_model.columns = ['id','topicDistributions','name','ThaKrowdScore']
dubai_model = dubai_model[['id','name','topicDistributions','ThaKrowdScore']]
dubai_model.head()

(8848, 2)


,id,name,topicDistributions,ThaKrowdScore
0,service_befa5188461d4caa8ef53d953548ef71,The Daily - Rove Healthcare City,"[4.903e-07, 0.012874757, 1.6109e-06, 6.796e-07...",2.302790
1,service_3e9b4ed6b1e84f79b5fc530d8722ba37,Pizza Hut,"[5.223e-07, 2.7402e-06, 1.7159e-06, 7.24e-07, ...",0.478185
2,service_51e6fc75fc2141a79be4483287949b2d,Yamanote Atelier,"[4.812e-07, 2.5246e-06, 1.5809e-06, 6.67e-07, ...",3.576049
3,service_21219562f6364cb195daaf28c686605e,Lichy Cafe,"[4.026e-07, 2.1121e-06, 1.3226e-06, 5.58e-07, ...",0.000000
4,service_0599883f41aa4839969147cf8f06d87a,Punjab Sarhad,"[4.841e-07, 2.5397e-06, 1.5904e-06, 6.71e-07, ...",0.439133


In [20]:
dubai_model.set_index('id',inplace=True)
dubai_model.head()

,name,topicDistributions,ThaKrowdScore
id,,,
service_befa5188461d4caa8ef53d953548ef71,The Daily - Rove Healthcare City,"[4.903e-07, 0.012874757, 1.6109e-06, 6.796e-07...",2.302790
service_3e9b4ed6b1e84f79b5fc530d8722ba37,Pizza Hut,"[5.223e-07, 2.7402e-06, 1.7159e-06, 7.24e-07, ...",0.478185
service_51e6fc75fc2141a79be4483287949b2d,Yamanote Atelier,"[4.812e-07, 2.5246e-06, 1.5809e-06, 6.67e-07, ...",3.576049
service_21219562f6364cb195daaf28c686605e,Lichy Cafe,"[4.026e-07, 2.1121e-06, 1.3226e-06, 5.58e-07, ...",0.000000
service_0599883f41aa4839969147cf8f06d87a,Punjab Sarhad,"[4.841e-07, 2.5397e-06, 1.5904e-06, 6.71e-07, ...",0.439133


In [21]:
with open(london_file) as f:
    data = ast.literal_eval(f.read())
london_model = pd.DataFrame(data)
print(london_model.shape)
london_model = london_model.merge(london_entity_df,on='id',how='inner')
london_model.columns = ['id','topicDistributions','name','ThaKrowdScore']
london_model = london_model[['id','name','topicDistributions','ThaKrowdScore']]
london_model.head()

(12121, 2)


,id,name,topicDistributions,ThaKrowdScore
0,service_15496525315504ymk5q6sk1k,The Thames,"[4.951e-07, 2.5973e-06, 1.6264e-06, 6.862e-07,...",2.334134
1,service_15496520333351tni4q3a6b9,Asado Steakhouse,"[4.608e-07, 2.4177e-06, 1.514e-06, 6.388e-07, ...",2.762054
2,service_1549654489597k35evbtfmnk,La Piazzetta Bistro & Bar,"[4.983e-07, 2.6143e-06, 1.6371e-06, 6.907e-07,...",1.333593
3,service_1549650327042ssjmpgyvood,German Gymnasium,"[4.97e-07, 2.6073e-06, 1.6327e-06, 6.889e-07, ...",3.741667
4,service_1549654281612y5o71a9qjpa,Wasabi Bento & Sushi,"[5.084e-07, 2.6671e-06, 1.6701e-06, 7.046e-07,...",1.084203


In [22]:
london_model.set_index('id',inplace=True)
london_model.head()

,name,topicDistributions,ThaKrowdScore
id,,,
service_15496525315504ymk5q6sk1k,The Thames,"[4.951e-07, 2.5973e-06, 1.6264e-06, 6.862e-07,...",2.334134
service_15496520333351tni4q3a6b9,Asado Steakhouse,"[4.608e-07, 2.4177e-06, 1.514e-06, 6.388e-07, ...",2.762054
service_1549654489597k35evbtfmnk,La Piazzetta Bistro & Bar,"[4.983e-07, 2.6143e-06, 1.6371e-06, 6.907e-07,...",1.333593
service_1549650327042ssjmpgyvood,German Gymnasium,"[4.97e-07, 2.6073e-06, 1.6327e-06, 6.889e-07, ...",3.741667
service_1549654281612y5o71a9qjpa,Wasabi Bento & Sushi,"[5.084e-07, 2.6671e-06, 1.6701e-06, 7.046e-07,...",1.084203


In [23]:
print(len(london_model.loc['service_1549649854375pj40ofwy2g','topicDistributions']))
print(len(dubai_model.loc['service_02de5d9900074921bb53cb698558f3e4','topicDistributions']))

180
180


# 3 inputs here from london and output from dubai

In [48]:
output_model = dubai_model[dubai_model.ThaKrowdScore>=dubai_model.ThaKrowdScore.quantile(q=0)]
#output_model = london_model[london_model.ThaKrowdScore>=london_model.ThaKrowdScore.quantile(q=0)]

In [49]:
#input_df = london_model[london_model.index.isin(['service_1549650727358zlpgs3llw3','service_15496498137596lk8dw2iu8h','service_1549651086285jmuzf9lqnn'])]
input_df = dubai_model[dubai_model.index.isin(['service_50125b740b20426ba5cbed87df0f43c4','service_5613c14b95db46788b962f4198c10009','service_4111729d985442998b7042e6ad0930f8'])]

In [50]:
input_df

,name,topicDistributions,ThaKrowdScore
id,,,
service_50125b740b20426ba5cbed87df0f43c4,Tresind,"[4.904e-07, 2.5729e-06, 1.6111e-06, 6.798e-07,...",4.219307
service_5613c14b95db46788b962f4198c10009,Via Delhi,"[4.797e-07, 2.5164e-06, 1.5758e-06, 6.648e-07,...",2.753051
service_4111729d985442998b7042e6ad0930f8,Nobu - Atlantis The Palm,"[4.932e-07, 2.5873e-06, 1.6202e-06, 0.00052554...",3.210166


In [51]:
inputlist = list(input_df.index)
inputlist

['service_50125b740b20426ba5cbed87df0f43c4',
 'service_5613c14b95db46788b962f4198c10009',
 'service_4111729d985442998b7042e6ad0930f8']

In [52]:
def populateJSDWithID(inputlist,input_df,output_df):
    for item in inputlist:
        col_name =input_df.loc[item,'name']+'-jsd'
        output_df[col_name] = (list(map(lambda x:jsd(input_df.loc[item,'topicDistributions'],x) ,output_df.topicDistributions)))
    return output_df

In [53]:
result_df = populateJSDWithID(inputlist,input_df,output_model)
result_df.head()

,name,topicDistributions,ThaKrowdScore,Tresind-jsd,Via Delhi-jsd,Nobu - Atlantis The Palm-jsd
id,,,,,,
service_befa5188461d4caa8ef53d953548ef71,The Daily - Rove Healthcare City,"[4.903e-07, 0.012874757, 1.6109e-06, 6.796e-07...",2.302790,0.373152,0.398448,0.384461
service_3e9b4ed6b1e84f79b5fc530d8722ba37,Pizza Hut,"[5.223e-07, 2.7402e-06, 1.7159e-06, 7.24e-07, ...",0.478185,0.660487,0.585140,0.606867
service_51e6fc75fc2141a79be4483287949b2d,Yamanote Atelier,"[4.812e-07, 2.5246e-06, 1.5809e-06, 6.67e-07, ...",3.576049,0.341394,0.486910,0.263681
service_21219562f6364cb195daaf28c686605e,Lichy Cafe,"[4.026e-07, 2.1121e-06, 1.3226e-06, 5.58e-07, ...",0.000000,0.685963,0.692803,0.692628
service_0599883f41aa4839969147cf8f06d87a,Punjab Sarhad,"[4.841e-07, 2.5397e-06, 1.5904e-06, 6.71e-07, ...",0.439133,0.666388,0.617287,0.626622


In [54]:
for item in list(input_df.name):
    result_df[item+'-score'] = result_df['ThaKrowdScore']/result_df[item+'-jsd']

In [55]:
result_df.head()

,name,topicDistributions,ThaKrowdScore,Tresind-jsd,Via Delhi-jsd,Nobu - Atlantis The Palm-jsd,Tresind-score,Via Delhi-score,Nobu - Atlantis The Palm-score
id,,,,,,,,,
service_befa5188461d4caa8ef53d953548ef71,The Daily - Rove Healthcare City,"[4.903e-07, 0.012874757, 1.6109e-06, 6.796e-07...",2.302790,0.373152,0.398448,0.384461,6.171180,5.779397,5.989655
service_3e9b4ed6b1e84f79b5fc530d8722ba37,Pizza Hut,"[5.223e-07, 2.7402e-06, 1.7159e-06, 7.24e-07, ...",0.478185,0.660487,0.585140,0.606867,0.723988,0.817214,0.787956
service_51e6fc75fc2141a79be4483287949b2d,Yamanote Atelier,"[4.812e-07, 2.5246e-06, 1.5809e-06, 6.67e-07, ...",3.576049,0.341394,0.486910,0.263681,10.474831,7.344380,13.562013
service_21219562f6364cb195daaf28c686605e,Lichy Cafe,"[4.026e-07, 2.1121e-06, 1.3226e-06, 5.58e-07, ...",0.000000,0.685963,0.692803,0.692628,0.000000,0.000000,0.000000
service_0599883f41aa4839969147cf8f06d87a,Punjab Sarhad,"[4.841e-07, 2.5397e-06, 1.5904e-06, 6.71e-07, ...",0.439133,0.666388,0.617287,0.626622,0.658975,0.711392,0.700794


In [56]:
result_df.reset_index(drop=False,inplace=True)

In [57]:
result_df = result_df[['id','name']+[item+'-jsd' for item in list(input_df.name)]+[item+'-score' for item in list(input_df.name)] ]

In [58]:
result_df.head()

,id,name,Tresind-jsd,Via Delhi-jsd,Nobu - Atlantis The Palm-jsd,Tresind-score,Via Delhi-score,Nobu - Atlantis The Palm-score
0,service_befa5188461d4caa8ef53d953548ef71,The Daily - Rove Healthcare City,0.373152,0.398448,0.384461,6.171180,5.779397,5.989655
1,service_3e9b4ed6b1e84f79b5fc530d8722ba37,Pizza Hut,0.660487,0.585140,0.606867,0.723988,0.817214,0.787956
2,service_51e6fc75fc2141a79be4483287949b2d,Yamanote Atelier,0.341394,0.486910,0.263681,10.474831,7.344380,13.562013
3,service_21219562f6364cb195daaf28c686605e,Lichy Cafe,0.685963,0.692803,0.692628,0.000000,0.000000,0.000000
4,service_0599883f41aa4839969147cf8f06d87a,Punjab Sarhad,0.666388,0.617287,0.626622,0.658975,0.711392,0.700794


In [59]:
result_df.to_csv("/home/krowd/Desktop/dubai-to-dubai-tresind-viadelhi-nobu-sampled.csv",header=True)

### Token-Distribution of a restaurant reviews